# M1M3 actuator movies
Craig Lage - 20-Apr-23 \
The 17 tons of mirror are supported by 156 pneumatic actuators where 44 are single-axis and provide support only on the axial direction, 100 are dual-axis providing support in the axial and lateral direction, and 12 are dual-axis providing support in the axial and cross lateral directions. \
Positioning is provided by 6 hard points in a hexapod configuration which moves the mirror to a fixed operational position that shall be maintained during telescope operations. The remaining optical elements will be moved relative to this position in order to align the telescope optics. Support and optical figure correction is provided by 112 dual axis and 44 single axis pneumatic actuators. 

In [ ]:
import sys, time, os, asyncio, glob
import shlex, subprocess
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
from astropy.time import Time, TimeDelta
from lsst.ts.xml.tables.m1m3 import FATable, FAIndex, force_actuator_from_id, actuator_id_to_index
from lsst_efd_client import EfdClient


## Set up the necessary subroutines

In [ ]:
def actuatorLayout(ax, FATable):
    ax.set_xlabel("X position (m)")
    ax.set_ylabel("Y position (m)")
    ax.set_title("M1M3 Actuator positions and type\nHardpoints are approximate", fontsize=18)
    types = [['SAA','NA', 'o', 'Z', 'b'], ['DAA','Y_PLUS', '^', 'Y_PLUS','g'], ['DAA','Y_MINUS', 'v', 'Y_MINUS', 'cyan'], \
             ['DAA','X_PLUS', '>', 'X_PLUS', 'r'], ['DAA','X_MINUS', '<', 'X_MINUS', 'r']]
    for [type, orient, marker, label, color] in types:
        xs = []
        ys = []
        for i in range(len(FATable)):
            x = FATable[i].x_position
            y = FATable[i].y_position
            if FATable[i].actuator_type.name == type and FATable[i].orientation.name == orient:
                xs.append(x)
                ys.append(y)
            else:
                continue
        ax.scatter(xs, ys, marker=marker, color=color, s=200, label=label)

    # Now plot approximate hardpoint location
    Rhp = 3.1 # Radius in meters
    for i in range(6):
        theta = 2.0 * np.pi / 6.0 * float(i)
        if i == 0:
            ax.scatter(Rhp * np.cos(theta), Rhp * np.sin(theta), marker='o', color='magenta', \
                       s=200, label='HP')
        else:
            ax.scatter(Rhp * np.cos(theta), Rhp * np.sin(theta), marker='o', color='magenta', \
                       s=200, label='_nolegend_')
    ax.legend(loc='lower left', fontsize=9)
    
    
def barChartZ(df, df_zero, ax, FATable, index, zmin, zmax):
    ax.set_xlabel("X position (m)")
    ax.set_ylabel("Y position (m)")
    ax.set_zlabel("Force (nt)")
    ax.set_title("M1M3 Actuator Z forces", fontsize=18)

    lightsource = LightSource(azdeg=180, altdeg=78)
    greyColor = '0.9'
    colors = []
    xs = []
    ys = []
    for i in range(len(FATable)):
        x = FATable[i].x_position
        y = FATable[i].y_position
        xs.append(x)
        ys.append(y)
        if FATable[i].actuator_type.name == 'SAA':
            colors.append('blue'); colors.append('blue')
            colors.append(greyColor); colors.append(greyColor)
            colors.append(greyColor); colors.append(greyColor)
        else:
            if FATable[i].orientation.name in ['Y_PLUS', 'Y_MINUS']:
                colors.append('green'); colors.append('green')
                colors.append(greyColor); colors.append(greyColor)
                colors.append(greyColor); colors.append(greyColor)
            if FATable[i].orientation.name in ['X_PLUS', 'X_MINUS']:
                colors.append('red'); colors.append('red')
                colors.append(greyColor); colors.append(greyColor)
                colors.append(greyColor); colors.append(greyColor)

    zs = np.zeros([len(FATable)])
    for i in range(len(FATable)):
        name=f"zForce{i}"
        zs[i] = df.iloc[index][name] - df_zero.iloc[0][name]

    dxs = 0.2 * np.ones([len(FATable)])
    dys = 0.2 * np.ones([len(FATable)])
    bottom = np.zeros([len(FATable)])
    ax.bar3d(xs, ys, bottom, dxs, dys, zs, shade=True, alpha=0.5, lightsource=lightsource, color=colors)
    ax.set_zlim(zmin, zmax)
    ax.view_init(elev=30., azim=225)
    

def heatMapZ(df, df_zero, ax, FATable, index, zmin, zmax):
    ax.set_xlabel("X position (m)")
    ax.set_ylabel("Y position (m)")
    ax.set_title("M1M3 Actuator Z forces (nt)", fontsize=18)

    types = [['SAA','NA', 'o', 'Z'], ['DAA','Y_PLUS', '^', 'Y_PLUS'], ['DAA','Y_MINUS', 'v', 'Y_MINUS'], \
             ['DAA','X_PLUS', '>', 'X_PLUS'], ['DAA','X_MINUS', '<', 'X_MINUS']]

    for [type, orient, marker, label] in types:
        xs = []
        ys = []
        zs = []
        for i in range(len(FATable)):
            x = FATable[i].x_position
            y = FATable[i].y_position
            if FATable[i].actuator_type.name == type and FATable[i].orientation.name == orient:
                xs.append(x)
                ys.append(y)
                name=f"zForce{i}"
                zs.append(df.iloc[index][name] - df_zero.iloc[0][name])
        im = ax.scatter(xs, ys, marker=marker, c=zs, cmap='RdBu_r', vmin=zmin, vmax=zmax, s=50, label=label)
    plt.colorbar(im, ax=ax,fraction=0.055, pad=0.02, cmap='RdBu_r')  
    
    
    
def lateralForces(df, df_zero, ax, FATable, index, forceMax):
    ax.set_xlabel("X position (m)")
    ax.set_ylabel("Y position (m)")
    ax.set_title("M1M3 lateral forces (nt)", fontsize=18)
    ax.set_xlim(-4.5,4.5)
    ax.set_ylim(-4.5,4.5)
    types = [['DAA','Y_PLUS', '^', 'Y_PLUS','g'], ['DAA','Y_MINUS', 'v', 'Y_MINUS', 'cyan'], \
             ['DAA','X_PLUS', '>', 'X_PLUS', 'r'], ['DAA','X_MINUS', '<', 'X_MINUS', 'r']]
    for [type, orient, marker, label, color] in types:
        xs = []
        ys = []
        arrowXs = []
        arrowYs = []
        for i in range(len(FATable)):
            x = FATable[i].x_position
            y = FATable[i].y_position
            if FATable[i].actuator_type.name == type and FATable[i].orientation.name == orient:
                xs.append(x)
                ys.append(y)
                if orient == 'X_PLUS':
                    name = f"xForce{FATable[i].x_index}"
                    arrowXs.append(df.iloc[index][name] / forceMax)
                    arrowYs.append(0.0)
                if orient == 'X_MINUS':
                    name = f"xForce{FATable[i].x_index}"
                    arrowXs.append(-df.iloc[index][name] / forceMax)
                    arrowYs.append(0.0)
                if orient == 'Y_PLUS':
                    name = f"yForce{FATable[i].y_index}"
                    arrowXs.append(0.0)
                    arrowYs.append(df.iloc[index][name] / forceMax)
                if orient == 'Y_MINUS':
                    name = f"yForce{FATable[i].y_index}"
                    arrowXs.append(0.0)
                    arrowYs.append(-df.iloc[index][name] / forceMax)
            else:
                continue
        ax.scatter(xs, ys, marker=marker, color=color, s=50, label=label)
        for ii in range(len(xs)):
            ax.arrow(xs[ii], ys[ii], arrowXs[ii], arrowYs[ii], color=color)

    ax.plot([-4.0,-3.0], [-4.0,-4.0], color='g')
    ax.text(-4.0, -4.3, f"{forceMax} nt")


def getZeroValuesAndLimits(df, subtractBaseline, t0, t1):
    # First define the zero values
    df_zero = df.head(1)
    for column_name in df_zero.columns:
        try:
            if subtractBaseline:
                df_zero.iloc[0, df_zero.columns.get_loc(column_name)] = np.median(df[column_name].values[t0:t1])
            else:
                df_zero.iloc[0, df_zero.columns.get_loc(column_name)] = 0.0
        except:
            continue
    # Now calculate the limits 
    types = [['SAA','NA'], ['DAA','Y_PLUS'], ['DAA','Y_MINUS'], ['DAA','X_PLUS'], ['DAA','X_MINUS']]
    zmin = 0.0; ymin = 0.0; xmin = 0.0; zmax = 0.0; ymax = 0.0; xmax = 0.0
    for [type, orient] in types:
            for i in range(len(FATable)):
                if type == 'SAA':
                    name = f"zForce{FATable[i].z_index}"
                    zmin = min(zmin, np.min(df[name] - df_zero.iloc[0][name]))                
                    zmax = max(zmax, np.max(df[name] - df_zero.iloc[0][name]))
                if orient in ['Y_PLUS', 'Y_MINUS']:
                    index = FATable[i].y_index
                    if index:
                        name = f"yForce{FATable[i].y_index}"
                        ymin = min(ymin, np.min(df[name] - df_zero.iloc[0][name]))                
                        ymax = max(ymax, np.max(df[name] - df_zero.iloc[0][name]))
                if orient in ['X_PLUS', 'X_MINUS']:
                    index = FATable[i].x_index
                    if index:
                        name = f"xForce{FATable[i].x_index}"
                        xmin = min(xmin, np.min(df[name] - df_zero.iloc[0][name]))                
                        xmax = max(xmax, np.max(df[name] - df_zero.iloc[0][name]))

    lateralMax = max(xmax, ymax, ymin, ymin)
    return [round(zmin), round(zmax), round(lateralMax), df_zero]


## Define the times and options

In [ ]:
# Times to start looking at encoder values
start = Time("2023-11-16T06:15:00", scale='utc')
end = Time("2023-11-16T06:30:00", scale='utc')

autoScale = True
# The following are only used if autoScale = False
zmin = 0.0
zmax = 2000.0
lateralMax = 1500.0

# The following average the first 100 data points
# and subtract these from the measurements
# If subtractBaseline = False, the unmodified values will be plotted
subtractBaseline = True
baselineT0 = 0.0
baselineT1 = 100.0

# The following allows you to plot only every nth data point
# If this value is 1, a frame will be made for every data point
# Of course, this takes longer
# If this value is 50, it will make a frame every second
frameN = 50

## Now generate the frames
### This will take some time

In [ ]:
client = EfdClient('usdf_efd')
timestamp = start.isot.split('.')[0].replace('-','').replace(':','')
dirName = f"/home/c/cslage/u/MTM1M3/movies/movie_{timestamp}"
%mkdir -p {dirName}
    
forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", "*", \
                                         start, end)
[autoZmin, autoZmax, autoLateralMax, forces_zero] = \
    getZeroValuesAndLimits(forces, subtractBaseline, baselineT0, baselineT1)
print(len(forces), len(forces_zero))
if autoScale:
    zmin = autoZmin
    zmax = autoZmax
    lateralMax = autoLateralMax

# Build the individual frames
fig = plt.figure(figsize=(16,16))
for n in range(0, len(forces), frameN):
    ax1 = fig.add_subplot(2,2,1)
    actuatorLayout(ax1, FATable)
    ax2 = fig.add_subplot(2,2,2, projection='3d')
    barChartZ(forces, forces_zero, ax2, FATable, n, zmin, zmax)
    ax3 = fig.add_subplot(2,2,3)
    lateralForces(forces, forces_zero, ax3, FATable, n, lateralMax)
    ax4 = fig.add_subplot(2,2,4)
    heatMapZ(forces, forces_zero, ax4, FATable, n, zmin, zmax)
    plt.savefig(f"{dirName}/Frame_{n:05d}.png")
    plt.clf()

## Now build the movie

In [ ]:
print(f"\033[1mThe movie name will be: {dirName}/m1m3_movie.mp4\033[0m")

command = f"ffmpeg -pattern_type glob -i '{dirName}/*.png' -f mp4 -vcodec libx264 -pix_fmt yuv420p -framerate 50 -y {dirName}/m1m3_movie.mp4"
args = shlex.split(command)
build_movie = subprocess.Popen(args)
build_movie.wait()